<a href="https://colab.research.google.com/github/tanoManzo/mimic_attitude/blob/dev/zero_shot_mimic_sentiment_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zero-shot Models Evaluation & Relabeling 


load and explore

In [ ]:
import pandas as pd
import numpy as np

load annotations

In [ ]:
path_annotations = '/content/drive/MyDrive/NIH/Data/annotated_burnout/anno.jsonl'
df_notes = df_notes.iloc[20:][['id','text','cats']].reset_index(drop=True)
df_notes

,id,text,cats
0,670,NPN addendum\n\n\n4. Completed 21day course of...,[Overall Positive Note]
1,671,Nursing NICU Note\n\n\nPt required frequent ba...,[Overall Negative Note]
2,672,Nursing NICU Note\n\n\n1. resp. O/Pt remains o...,[Overall Neutral Note]
3,673,Nursing progress note\n\n\n#1 O: [**Known last...,[Overall Positive Note]
4,674,NP NOTE\nProcedure Note: [** 1876**] removal\n...,[Overall Neutral Note]
...,...,...,...
218,888,NPN 1900-0700\n\n\n2. RESP: Infant remains ...,[Overall Neutral Note]
219,889,NICU NSG ADMIT NOTE\n\n28 [**1-2**] week femal...,[Overall Neutral Note]
220,890,Nursing NICU Note Cont.\n\n6 Alt. in CV status...,[Overall Neutral Note]
221,891,"NPN Days\n\n\n#1 ID: Temp stable, infant is ac...",[Overall Neutral Note]


load results from models outputs

In [ ]:
results_label = pd.read_json('/content/drive/MyDrive/NIH/Data/results_burnout/label_zeroshot.json')
results_label

,roberta,distilbert,minilm,bloom
0,neutral,NEGATIVE,LABEL_0,LABEL_0
1,neutral,NEGATIVE,LABEL_0,LABEL_0
2,neutral,NEGATIVE,LABEL_0,LABEL_0
3,neutral,NEGATIVE,LABEL_0,LABEL_0
4,neutral,NEGATIVE,LABEL_0,LABEL_1
...,...,...,...,...
187,neutral,NEGATIVE,LABEL_0,LABEL_0
188,neutral,NEGATIVE,LABEL_0,LABEL_0
189,neutral,NEGATIVE,LABEL_0,LABEL_0
190,neutral,NEGATIVE,LABEL_0,LABEL_0


explore

In [ ]:
for col in results_label.columns:
  print(results_label[col].value_counts(sort=True), '\n')

neutral     189
negative      2
positive      1
Name: roberta, dtype: int64 

NEGATIVE    177
POSITIVE     15
Name: distilbert, dtype: int64 

LABEL_0    192
Name: minilm, dtype: int64 

LABEL_0    159
LABEL_1     33
Name: bloom, dtype: int64 



pre-processing basic, mapping same sentiment 

In [ ]:
num_notes = 192
labels = {}

In [ ]:
def mapping_origin(labels):
  mapping = {'Overall Positive Note':'positive', 'Overall Negative Note':'negative', 'Overall Neutral Note':'neutral'}
  return labels.map(lambda x: mapping[x])

true_label = mapping_origin(df_notes.iloc[:num_notes]['cats'].map(lambda x:x[0]))

In [ ]:
labels['roberta_label'] = results_label['roberta']

In [ ]:
def mapping_distil_origin(labels):
  mapping = {'NEGATIVE':'negative', 'POSITIVE':'positive'}
  return labels.map(lambda x: mapping[x])

labels['distil_label']  = mapping_distil_origin(results_label['distilbert'])

In [ ]:
def mapping_distil_origin(labels):
  mapping = {'LABEL_0':'negative', 'LABEL_1':'positive'}
  return labels.map(lambda x: mapping[x])

labels['minilm_label'] = mapping_distil_origin(results_label['minilm'])

In [ ]:
def mapping_distil_origin(labels):
  mapping = {'LABEL_0':'negative', 'LABEL_1':'positive'}
  return labels.map(lambda x: mapping[x])

labels['bloom_label'] = mapping_distil_origin(results_label['bloom'])

evaluation basic 

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

for name_pred, y_pred in labels.items():
  print(name_pred)
  print(precision_recall_fscore_support(true_label, y_pred, average='macro'))
  print(precision_recall_fscore_support(true_label, y_pred, average='micro'))
  print(precision_recall_fscore_support(true_label, y_pred, average='weighted'))
  print(accuracy_score(true_label, y_pred))
  print(accuracy_score(true_label, y_pred, normalize=False))

roberta_label
(0.753968253968254, 0.3683360700602079, 0.35672521900066806, None)
(0.7604166666666666, 0.7604166666666666, 0.7604166666666666, None)
(0.7290426587301587, 0.7604166666666666, 0.6706665930094072, None)
0.7604166666666666
146
distil_label
(0.1713747645951036, 0.44365079365079363, 0.22408572094106685, None)
(0.19270833333333334, 0.19270833333333334, 0.19270833333333331, None)
(0.05379001883239171, 0.19270833333333334, 0.07817959468902864, None)
0.19270833333333334
37
minilm_label
(0.06076388888888889, 0.3333333333333333, 0.10279001468428779, None)
(0.18229166666666666, 0.18229166666666666, 0.18229166666666663, None)
(0.03323025173611111, 0.18229166666666666, 0.05621328928046989, None)
0.18229166666666666
35
bloom_label
(0.10920526014865638, 0.4055555555555556, 0.17029400534555172, None)
(0.171875, 0.171875, 0.171875, None)
(0.041571374118543926, 0.171875, 0.06650916380297824, None)
0.171875
33


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classificat

pre-process ad-hoc, generate neutral

In [ ]:
score_label = pd.read_json('/content/drive/MyDrive/NIH/Data/results_burnout/score_zeroshot.json')
score_label.describe()

,roberta,distilbert,minilm,bloom
count,192.000000,192.000000,192.000000,192.000000
mean,0.827485,0.968404,0.501422,0.992690
std,0.080960,0.080818,0.000362,0.040856
min,0.495300,0.561700,0.500300,0.601400
25%,0.805575,0.989625,0.501300,1.000000
50%,0.850050,0.995700,0.501500,1.000000
75%,0.875700,0.997925,0.501700,1.000000
max,0.938100,0.999500,0.502100,1.000000


In [ ]:
labels_adhoc = labels

In [ ]:
def get_outlier_index(items,column):
  df = items[column]
  q1 = df.quantile(0.25)
  q3 = df.quantile(0.75)
  IQR=q3-q1
  lower = df[(df<(q1-1.5*IQR))]
  return lower.index

labels_adhoc['distil_label'][get_outlier_index(score_label,'distilbert')]='neutral'
labels_adhoc['bloom_label'][get_outlier_index(score_label,'bloom')]='neutral'

In [ ]:
labels_adhoc['distil_label'].value_counts()

negative    154
neutral      34
positive      4
Name: distilbert, dtype: int64

In [ ]:
labels_adhoc['bloom_label'].value_counts()

negative    154
neutral      35
positive      3
Name: bloom, dtype: int64

evaluation ad hoc

In [ ]:
for name_pred, y_pred in labels_adhoc.items():
  print(name_pred)
  print(precision_recall_fscore_support(true_label, y_pred, average='macro'))
  print(precision_recall_fscore_support(true_label, y_pred, average='micro'))
  print(precision_recall_fscore_support(true_label, y_pred, average='weighted'))
  print(accuracy_score(true_label, y_pred))
  print(accuracy_score(true_label, y_pred, normalize=False))

roberta_label
(0.753968253968254, 0.3683360700602079, 0.35672521900066806, None)
(0.7604166666666666, 0.7604166666666666, 0.7604166666666666, None)
(0.7290426587301587, 0.7604166666666666, 0.6706665930094072, None)
0.7604166666666666
146
distil_label
(0.46473134708428826, 0.38691844553913524, 0.27501157715310415, None)
(0.2864583333333333, 0.2864583333333333, 0.2864583333333333, None)
(0.5986658868092692, 0.2864583333333333, 0.27408032450513825, None)
0.2864583333333333
55
minilm_label
(0.06076388888888889, 0.3333333333333333, 0.10279001468428779, None)
(0.18229166666666666, 0.18229166666666666, 0.18229166666666663, None)
(0.03323025173611111, 0.18229166666666666, 0.05621328928046989, None)
0.18229166666666666
35
bloom_label
(0.3907647907647907, 0.34731800766283527, 0.22839506172839505, None)
(0.2708333333333333, 0.2708333333333333, 0.2708333333333333, None)
(0.5502570346320347, 0.2708333333333333, 0.25534336419753095, None)
0.2708333333333333
52


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


pre-processing collapse, negative into positive

In [ ]:
true_label_collapse = true_label
true_label_collapse[true_label_collapse =='negative'] = 'positive'
true_label_collapse.value_counts()

neutral     145
positive     47
Name: cats, dtype: int64

In [ ]:
labels_collapse = labels_adhoc
labels_collapse['distil_label'][labels_collapse['distil_label'] =='negative'] = 'positive'
labels_collapse['bloom_label'][labels_collapse['bloom_label'] =='negative'] = 'positive'
labels_collapse['roberta_label'][labels_collapse['roberta_label'] =='negative'] = 'positive'

In [ ]:
labels_collapse['bloom_label'].value_counts()

positive    157
neutral      35
Name: bloom, dtype: int64

evaluation collapse 

In [ ]:

for name_pred, y_pred in labels_collapse.items():
  print(name_pred)
  print(precision_recall_fscore_support(true_label_collapse, y_pred, average='macro'))
  print(precision_recall_fscore_support(true_label_collapse, y_pred, average='micro'))
  print(precision_recall_fscore_support(true_label_collapse, y_pred, average='weighted'))
  print(accuracy_score(true_label_collapse, y_pred))
  print(accuracy_score(true_label_collapse, y_pred, normalize=False))

roberta_label
(0.7142857142857142, 0.5178283198826119, 0.47113772455089814, None)
(0.7604166666666666, 0.7604166666666666, 0.7604166666666666, None)
(0.7385912698412698, 0.7604166666666666, 0.6707809381237525, None)
0.7604166666666666
146
distil_label
(0.470029784065525, 0.47637564196625093, 0.3145660171685516, None)
(0.3177083333333333, 0.3177083333333333, 0.3177083333333333, None)
(0.5904128660958055, 0.3177083333333333, 0.2908777876186583, None)
0.3177083333333333
61
minilm_label
(0.0, 0.0, 0.0, None)
(0.0, 0.0, 0.0, None)
(0.0, 0.0, 0.0, None)
0.0
0
bloom_label
(0.44003639672429484, 0.4516507703595011, 0.2993464052287582, None)
(0.3020833333333333, 0.3020833333333333, 0.3020833333333333, None)
(0.550851152562936, 0.3020833333333333, 0.27699482570806105, None)
0.3020833333333333
58


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1